# Machine Learning to predict Microsoft Stock Prices with ML.NET

Uses the daily close of Microsoft's stock to train a model and predict stock prices based on the last 100 days.

## Technologies used

- [ML.NET](https://dotnet.microsoft.com/apps/machinelearning-ai/ml-dotnet) Machine Learning for .NET
- [.NET Interactive Notebooks](https://github.com/dotnet/interactive) Jupyter Notebooks for .NET Languages
- [XPlot](https://fslab.org/XPlot//index.html) F# Data Visualization Package

## Setup

Import required NuGet packages, setup `using` statements and create formatters for the data.

In [ ]:
#i "nuget:https://pkgs.dev.azure.com/dnceng/public/_packaging/dotnet5/nuget/v3/index.json" 
#i "nuget:https://pkgs.dev.azure.com/dnceng/public/_packaging/dotnet-tools/nuget/v3/index.json" 

#r "nuget:Microsoft.ML,1.5.5"
#r "nuget:Microsoft.ML.AutoML,0.17.5"
#r "nuget:Microsoft.ML.TimeSeries,1.5.5"
#r "nuget:Microsoft.Data.Analysis,0.4.0"
#r "nuget:XPlot.Plotly.Interactive,4.0.1"

Restore sources https://pkgs.dev.azure.com/dnceng/public/_packaging/dotnet5/nuget/v3/index.json

Restore sources https://pkgs.dev.azure.com/dnceng/public/_packaging/dotnet5/nuget/v3/index.json https://pkgs.dev.azure.com/dnceng/public/_packaging/dotnet-tools/nuget/v3/index.json

Installed package Microsoft.Data.Analysis version 0.4.0

Installed package Microsoft.ML.TimeSeries version 1.5.5

Installed package XPlot.Plotly.Interactive version 4.0.1

Installed package Microsoft.ML.AutoML version 0.17.5

Installed package Microsoft.ML version 1.5.5

Loading extensions from `XPlot.Plotly.Interactive.dll`

Configuring PowerShell Kernel for XPlot.Plotly integration.

Installed support for XPlot.Plotly.

In [ ]:
using static Microsoft.DotNet.Interactive.Formatting.PocketViewTags;
using Microsoft.DotNet.Interactive.Formatting;
using Microsoft.Data.Analysis;
using Microsoft.AspNetCore.Html;
using XPlot.Plotly;

In [ ]:
Formatter.Register<DataFrame>((df, writer) =>
{
    var headers = new List<IHtmlContent>();
    headers.Add(th(i("index")));
    headers.AddRange(df.Columns.Select(c => (IHtmlContent) th(c.Name)));
    var rows = new List<List<IHtmlContent>>();
    var take = 20;
    for (var i = 0; i < Math.Min(take, df.Rows.Count); i++)
    {
        var cells = new List<IHtmlContent>();
        cells.Add(td(i));
        foreach (var obj in df.Rows[i])
        {
            cells.Add(td(obj));
        }
        rows.Add(cells);
    }

    var t = table(
        thead(
            headers),
        tbody(
            rows.Select(
                r => tr(r))));

    writer.Write(t);
}, "text/html");

## Import Data

Load from local filesystem if it exists, otherwise download it

In [ ]:
using System.IO;
using System.Net.Http;
string stockPath = "msft.csv";

if (!File.Exists(stockPath))
{
    var contents = await new HttpClient()
      .GetStringAsync("https://query1.finance.yahoo.com/v7/finance/download/MSFT?period1=1458000000&period2=1915766400&interval=1d&events=history&includeAdjustedClose=true");

    File.WriteAllText(stockPath, contents);
}

var stockData = DataFrame.LoadCsv(stockPath);
stockData

0,2016-03-15,52.75,53.59,52.74,53.59,49.14052,21104800
1,2016-03-16,53.45,54.6,53.4,54.35,49.837418,31691700
2,2016-03-17,54.21,55,54,54.66,50.121674,28223900
3,2016-03-18,54.92,54.97,53.45,53.49,49.048813,67625500
4,2016-03-21,53.25,53.93,52.93,53.86,49.3881,23925700
5,2016-03-22,53.61,54.25,53.46,54.07,49.580658,23124100
6,2016-03-23,54.11,54.24,53.74,53.97,49.48897,20129000
7,2016-03-24,53.84,54.33,53.73,54.21,49.709045,19950000
8,2016-03-28,54.21,54.29,53.33,53.54,49.09467,17025100
9,2016-03-29,53.66,54.86,53.45,54.71,50.16752,23924300
10,2016-03-30,54.93,55.64,54.9,55.05,50.4793,23008300


## Explore the data

In [ ]:
var chart = Chart.Plot(
    new Scatter()
    {
        x = stockData.Columns["Date"],
        y = stockData.Columns["Adj Close"]
    }
);
display(chart);

<!DOCTYPE html>

## Split into Training and Test Data

In [ ]:
int testSize = (int)(stockData.Rows.Count * .1);

DataFrame stock_train = stockData.Head((int)stockData.Rows.Count - testSize);
DataFrame stock_test = stockData.Tail(testSize);

display(stock_train.Rows.Count);
display(stock_test.Rows.Count);

1145

127

In [ ]:
using Microsoft.ML;
using Microsoft.ML.Data;
using Microsoft.ML.AutoML;
using Microsoft.ML.Transforms;

In [ ]:
#!time

const int TRAINING_TIME = 15;

var mlContext = new MLContext();

var experiment = mlContext.Forecasting.ForecastBySsa(
    outputColumnName: "ForecastedPrice",
    inputColumnName: "Adj Close",
    windowSize: 100,
    seriesLength: 365,
    trainSize: (int)stock_train.Rows.Count,
    horizon: 7,
    confidenceLevel: 0.95f,
    confidenceLowerBoundColumn: "LowerBound",
    confidenceUpperBoundColumn: "UpperBound"
);

var forecaster = experiment.Fit(stock_train);
forecaster

Microsoft.ML.Transforms.TimeSeries.SsaForecastingTransformer

Wall time: 102.1381ms